In [1]:
# Importe
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

# Dateipfad
url = "https://raw.githubusercontent.com/hon3ypi/DashboardDesignFS23/main/newDataset.csv"

# Daten aus CSV-Datei laden
datafile = pd.read_csv(url)

# Daten für die ausgewählten Länder und Jahre filtern
countries = ['Switzerland', 'Germany', 'France', 'Austria', 'Italy', 'Liechtenstein']
filtComb = datafile.query('GeoAreaName in @countries')
# Daten für das Bubble-Diagramm filtern
bubble_data = datafile[datafile['Year'] == 2019]

# Diagramm für Tab 1
scatfig = px.line(filtComb, x="Year", y="Value", color='GeoAreaName')
hisfig = px.histogram(filtComb, x="Year", y="Value", color='GeoAreaName', barmode='group')

# Diagramm für Tab 2
scatty1 = px.line(filtComb, x="Year", y="Value", color='GeoAreaName')
scatty2 = px.line(filtComb, x="Year", y="Value", color='GeoAreaName')

# Diagramm für Tab 3
bubfig = px.scatter(bubble_data, y='Value', size='Value', color='GeoAreaName', hover_data=['GeoAreaName'])


# Web-App
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout
app.layout = html.Div([
    dbc.Tabs([
        # TAB 1 -------------------------------------------------------------------------------------
        dbc.Tab(label='Switzerland and Neighbours', children=[
            html.H1("Proportion of women in managerial positions", style={'text-align': 'center'}),
            html.H4("Data for Switzerland, Austria, Germany and Italy", style={'text-align': 'center'}),

            dcc.RangeSlider(
                id='scatter-slider',
                min=datafile['Year'].min(),
                max=datafile['Year'].max(),
                value=[datafile['Year'].min(), datafile['Year'].max()],
                marks={str(value): str(value) for value in datafile['Year'].unique()},
                step=None
            ),

            dbc.Row([
                dbc.Col(width=6, children=[
                    html.Label("Histogram", style={'text-align': 'center'}),
                    dcc.Graph(id='hist-plot', figure=hisfig)
                ]),
                dbc.Col(width=6, children=[
                    html.Label("Select Country", style={'text-align': 'center'}),
                    dcc.Dropdown(
                        id='line-country-dropdown',
                        options=[{'label': color, 'value': color} for color in filtComb['GeoAreaName'].unique()],
                        value='Switzerland'
                    ),
                    dcc.Graph(id='line-plot', figure=scatfig)
                ]),
            ]),
        ]),
        # TAB 2 -------------------------------------------------------------------------------------
        dbc.Tab(label='Compare', children=[
            html.H1("Proportion of women in managerial positions", style={'text-align': 'center'}),
            html.H4("Compare (only Switzerland, France, Italy, Austria & Germany)", style={'text-align': 'center'}),
            dbc.Row([
                dbc.Col(className="col-6", children=[
                    html.H4("this", style={'text-align': 'center'}),
                    html.Label("Select Country", style={'text-align': 'center'}),
                    dcc.Dropdown(
                        id='scatty1-dropdown',
                        options=[{'label': color, 'value': color} for color in filtComb['GeoAreaName'].unique()],
                        value='Switzerland'
                    ),
                    dcc.Graph(id='scatty1-plot')
                ]),
                dbc.Col(className="col-6", children=[
                    html.H4("that", style={'text-align': 'center'}),
                    html.Label("Select Country", style={'text-align': 'center'}),
                    dcc.Dropdown(
                        id='scatty2-dropdown',
                        options=[{'label': color, 'value': color} for color in filtComb['GeoAreaName'].unique()],
                        value='Germany'
                    ),
                    dcc.Graph(id='scatty2-plot')
                ]),
            ]),
        ]),
        # TAB 3 -------------------------------------------------------------------------------------
        dbc.Tab(label='World 2019', children=[
            html.H1("Proportion of women in managerial positions", style={'text-align': 'center'}),
            html.H4("World data for 2019", style={'text-align': 'center'}),
            dcc.Graph(figure=bubfig),
        ])
    ])
])

# Callbacks

@app.callback(Output('hist-plot', 'figure'), Input('scatter-slider', 'value'))
def update_histogram_plot(value):
    filtered_data = filtComb[(filtComb['Year'] >= value[0]) & (filtComb['Year'] <= value[1])]
    fig = px.histogram(filtered_data, x="Year", y="Value", color='GeoAreaName', barmode='group')
    return fig

@app.callback(Output('line-plot', 'figure'), Input('line-country-dropdown', 'value'), Input('scatter-slider', 'value'))
def update_line_plot(color, value):
    filtered_data = filtComb[(filtComb['Year'] >= value[0]) & (filtComb['Year'] <= value[1])]
    fig = px.line(filtered_data, x="Year", y="Value", color='GeoAreaName')
    
    # Farben beibehalten und nur das ausgewählte Land anzeigen
    for i, data in enumerate(fig.data):
        if data.name == color:
            fig.data[i].visible = True
        else:
            fig.data[i].visible = False
    
    return fig


@app.callback(Output('scatty1-plot', 'figure'), Input('scatty1-dropdown', 'value'), Input('scatter-slider', 'value'))
def update_scatty1_plot(country, value):
    filtered_data = filtComb[(filtComb['GeoAreaName'] == country) & (filtComb['Year'] >= value[0]) & (filtComb['Year'] <= value[1])]
    fig = px.line(filtered_data, x="Year", y="Value", color='GeoAreaName', color_discrete_map={"Switzerland": "red", "Germany": "black", "Italy": "green", "France": "blue", "Austria": "pink"})
    return fig

@app.callback(Output('scatty2-plot', 'figure'), Input('scatty2-dropdown', 'value'), Input('scatter-slider', 'value'))
def update_scatty2_plot(country, value):
    filtered_data = filtComb[(filtComb['GeoAreaName'] == country) & (filtComb['Year'] >= value[0]) & (filtComb['Year'] <= value[1])]
    fig = px.line(filtered_data, x="Year", y="Value", color='GeoAreaName', color_discrete_map={"Switzerland": "red", "Germany": "black", "Italy": "green", "France": "blue", "Austria": "pink"})
    return fig

if __name__ == '__main__':
    app.run_server(debug=False)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
